**Distancia entre las propiedades y algun estacion de subte, tren o parada de colectivo**

Para calcular la distancia entre cada una de las propiedades y alguna estacion de subbte, tren o parada de colectivo hicimos los siguientes pasos:

* Seleccionamos una determinada zona, por ejemplo, el barrio de Palermo.
* Descargamos las bases de datos de estaciones de subte, de ferrocarriles y las paradas de colectivos de CABA.
* Definimos la funcion Haversine
* Para empezar, calculamos la distancia entre Palermo y subtes.
* PROBLEMA: 
Pudimos calcular la distancia usando las lon y lat de la propiedad 1 con la estacion de subte 1, y las coordenadas de la propiedad 2 con la estacion 2.

Pero, a lo que queremos llegar es que calcule la propiedad 1 con la estacion 1 y la estacion 2 y la estacion 3 etc. Luego, la propiedad 2 con la estacion 1 estacion 2 estacion 3 etc… Se entiende?

In [17]:
import pandas as pd
import numpy as np

**Palermo**

In [13]:
data = pd.read_csv("properati.csv", index_col=0)
palermo = data.loc[data.place_name == 'Palermo', : ]

desired_col= ['property_type', 'place_name', 'state_name', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title']

paler= palermo[desired_col]
por_nul = paler.isnull().sum()/paler.shape[0]*100
por_nul_sorted = por_nul.sort_values(ascending=False)

lat_max = paler.lat.max()
lat_min = paler.lat.min()
lon_max = paler.lon.max()
lon_min = paler.lon.min()

paler.shape
#por_nul_sorted

(2885, 19)

In [15]:
paler.head(2)

,property_type,place_name,state_name,lat,lon,price,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title
19,apartment,Palermo,Capital Federal,-34.580504,-58.405874,350000.0,USD,6175575.00,350000.0,104.0,96.0,3365.384615,3645.833333,NaN,3.0,NaN,http://www.properati.com.ar/15bq8_venta_depart...,Excelente semipiso al contra frente en Bulnes ...,"Bulnes y Libertador: espectacular pulmón, con ..."
21,apartment,Palermo,Capital Federal,-34.590926,-58.411665,270500.0,USD,4772837.25,270500.0,118.0,73.0,2292.372881,3705.479452,NaN,4.0,NaN,http://www.properati.com.ar/15bqd_venta_depart...,"EXCELENTE ZONA, MULTIPLES MEDIOS DE TRANSPORTE...",Departamento de 4 ambientes en Venta en Palermo


**Data sets externos**

* Agreguen toda otra información construida a partir de los datos originales (o incluso información externa) que consideren relevante y útil para resolver los objetivos planteados.

In [3]:
subte_data = "https://raw.githubusercontent.com/carlachka/grupo2dh/master/subte.csv"
ferro_data = "https://raw.githubusercontent.com/carlachka/grupo2dh/master/ferrocarril.csv"
paradas_df = "https://raw.githubusercontent.com/carlachka/grupo2dh/master/paradas.csv"

subte = pd.read_csv(subte_data) 
ferro = pd.read_csv(ferro_data)
colec = pd.read_csv(paradas_df)


In [4]:
#Cambio los nombres de las columnas de paradas de colectivos asi los tres data sets tienen los mismos nombres
colec = colec.rename({
    'stop_lat': 'lat',
    'stop_lon': 'long',
}, axis=1)

In [11]:
subte.head(3) # long lat id estacion linea AGREGAR subte
colec.head(3) # lat long stop_id route_id  AGREGAR colectivo
ferro.head(3) # lat long id nombre barrio  AGREGAR ferrocarril
print(subte.shape)
print(colec.shape)
print(ferro.shape)

(90, 5)
(42463, 12)
(230, 11)


**Subte**

In [23]:
subte.head(3)

,long,lat,id,estacion,linea
0,-58.398928,-34.635750,1.0,CASEROS,H
1,-58.400970,-34.629376,2.0,INCLAN - MEZQUITA AL AHMAD,H
2,-58.402323,-34.623092,3.0,HUMBERTO 1°,H


**Distancia entre dos puntos de latitud y longitud - Haversine_formula**

The [haversine](https://en.wikipedia.org/wiki/Haversine_formula) formula allows the haversine of Θ (that is, hav(Θ)) to be computed directly from the latitude and longitude of the two points.

<img src="h_formula.png" alt="Drawing" style="width: 400px;"/>


* φ1, φ2 are the latitude of point 1 and latitude of point 2 (in radians),
* λ1, λ2 are the longitude of point 1 and longitude of point 2 (in radians).

As similar formula can be written using cosines (sometimes called the spherical law of cosines, not to be confused with the law of cosines for plane geometry) instead of haversines, but if the two points are close together (e.g. a kilometer apart, on the Earth) you might end up with an inaccurate answer. Since the haversine formula uses sines, it avoids that problem.

A [simplified formula](http://www.movable-type.co.uk/scripts/latlong.html) says:

<img src="h_simple.png" alt="Drawing" style="width: 600px;"/>


In [16]:
#vectorized numpy version of the same function

import numpy as np

def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    
    km = 6367 * c  #el radio de la tierra no es 6371 km??
    
    return km


Ejemplo de como usar la funcion

In [27]:
#The inputs are all arrays of values, and it should be able to do millions of points instantly. 
#The requirement is that the inputs are ndarrays but the columns of your pandas table will work.

#For example, with randomly generated values:

lon1, lon2, lat1, lat2 = np.random.randn(4, 1000000)
df = pd.DataFrame(data={'lon1':lon1,'lon2':lon2,'lat1':lat1,'lat2':lat2})
km = haversine_np(df['lon1'],df['lat1'],df['lon2'],df['lat2'])

#Or if you want to create another column:

df['distance'] = haversine_np(df['lon1'],df['lat1'],df['lon2'],df['lat2'])

#Looping through arrays of data is very slow in python. Numpy provides functions that operate on entire arrays of data, 
#which lets you avoid looping and drastically improve performance.
df.head(2)

,lon1,lon2,lat1,lat2,distance
0,0.556902,0.211185,0.480597,-1.996209,277.902736
1,0.410545,0.155902,0.428752,-0.974439,158.476321


**Calculo la distancia entre todas las propiedades de Palermo y las estaciones de subte**

In [33]:
#creo un nuevo df vacio
dist = pd.DataFrame() 

dist['distance'] = haversine_np(paler['lon'],paler['lat'],subte['long'],subte['lat'])
dist.head(25)

,distance
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [36]:
#PROBLEMA: deberia ser un loop, para que por cada (lon, lat) de las propiedades lo calcule con 
# CADA UNA de las estaciones. Como usar la funcion en forma de loop o crear un nuevo data set donde se repitan 
# las long y lat de las estaciones 2885 veces???

Lo anterior usa las lon y lat de la propiedad 1 con la estacion de subte 1, y las coordenadas de la propiedad 2 con la estacion 2.

Lo que quiero es que calcule la propiedad 1 con la estacion 1 y la estacion 2 y la estacion 3 etc. Luego, la propiedad 2 con la estacion 1 estacion 2 estacion 3 etc…Se entiende?